## Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('step0_results.csv', na_values=['no info', '.'], delimiter=',')

## Modify Table for Calculations

In [3]:
new_columns = ['date']

for i in range(1, 10):
    for j in range(1, 13):
        new_columns.append(str(i)+str(j))

new_columns.append('feedback')
df.columns = new_columns
df.drop(['date', 'feedback'], axis=1, inplace=True)

In [4]:
df.replace('Strongly disagree', 1, inplace=True)
df.replace('Disagree', 2, inplace=True)
df.replace('Neutral', 3, inplace=True)
df.replace('Agree', 4, inplace=True)
df.replace('Strongly Agree', 5, inplace=True)

df.replace('< 5 min', 1, inplace=True)
df.replace('5 - 15 min', 2, inplace=True)
df.replace('15 - 30 min', 3, inplace=True)
df.replace('30 - 60 min', 4, inplace=True)
df.replace('> 60 min', 5, inplace=True)

## Apply positive & negative

In [6]:
# multiplication of values with 1 or -1 depending on the question effect
for column in df:
    r = int(column)%10
    if r == 7 or r == 8 or r == 9 or r == 0:
        df[column] = df[column]*(-1)

## Sum points of each question

In [8]:
sumOverAll = np.nan
sumOverAll = df.sum(axis = 0)
#sumOverAll

In [9]:
final = np.empty([1,9], dtype=int)

for i in range(9):
    final[0,i] = sumOverAll[i*12:(i*12)+12].sum(axis=0)


array([[113,  72, 191, 161, 164, 147, 181, 180, 153]])

In [10]:
finalTable = pd.DataFrame(final, columns=['1', '2', '3', '4', '5', '6', '7', '8', '9'])
finalTable

,1,2,3,4,5,6,7,8,9
0,113,72,191,161,164,147,181,180,153
